# Topic : Image classification for color images
## Goal : Achieve high accuracy on image classification 
## Dataset : CIFAR-10 

## Step 1: Import the dependencies.

In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [17]:
import numpy as np
import matplotlib.pyplot as plt

from tensorflow.keras.datasets import cifar10
from tensorflow.keras.preprocessing import image
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras import optimizers, regularizers
from tensorflow.keras.layers import (Conv2D, Dense, BatchNormalization,
                                     Flatten, MaxPooling2D, Dropout, Activation)

## Step 2 : Get the data ready for training

– Download the data from the Keras library.

– Split it into train, validate, and test datasets.

– Normalize the data.

– One-hot encode the labels.

In [3]:
(images, labels), (X_test, y_test) =cifar10.load_data() # load the data set from keras

In [4]:
images.shape, X_test.shape, labels.shape, y_test.shape

((50000, 32, 32, 3), (10000, 32, 32, 3), (50000, 1), (10000, 1))

In [5]:
def split_data(data: np.ndarray, size : int) -> tuple:
    ''' this function splits a given dataset into 
    the training and validation set and returns a tuple'''
    return data[:size], data[size:] 
X_train, X_valid = split_data(images,40000 )
y_train, y_valid = split_data(labels, 40000)

In [6]:
X_train.shape, X_valid.shape, X_test.shape

((40000, 32, 32, 3), (10000, 32, 32, 3), (10000, 32, 32, 3))

In [7]:
X_train

array([[[[ 59,  62,  63],
         [ 43,  46,  45],
         [ 50,  48,  43],
         ...,
         [158, 132, 108],
         [152, 125, 102],
         [148, 124, 103]],

        [[ 16,  20,  20],
         [  0,   0,   0],
         [ 18,   8,   0],
         ...,
         [123,  88,  55],
         [119,  83,  50],
         [122,  87,  57]],

        [[ 25,  24,  21],
         [ 16,   7,   0],
         [ 49,  27,   8],
         ...,
         [118,  84,  50],
         [120,  84,  50],
         [109,  73,  42]],

        ...,

        [[208, 170,  96],
         [201, 153,  34],
         [198, 161,  26],
         ...,
         [160, 133,  70],
         [ 56,  31,   7],
         [ 53,  34,  20]],

        [[180, 139,  96],
         [173, 123,  42],
         [186, 144,  30],
         ...,
         [184, 148,  94],
         [ 97,  62,  34],
         [ 83,  53,  34]],

        [[177, 144, 116],
         [168, 129,  94],
         [179, 142,  87],
         ...,
         [216, 184, 140],
        

In [8]:
# normalizing the data
def normalize_data(dataSetSplit : np.ndarray)-> np.ndarray:
    mean = np.mean(X_train)
    std = np.std(X_train)
    return (dataSetSplit.astype('float32') - mean) / std
    
X_train = normalize_data(X_train)
X_valid = normalize_data(X_valid)
X_test = normalize_data(X_test)

In [9]:
X_train

array([[[[-0.96084046, -0.9141054 , -0.8985271 ],
         [-1.2100939 , -1.1633588 , -1.1789372 ],
         [-1.1010455 , -1.1322021 , -1.2100939 ],
         ...,
         [ 0.5814151 ,  0.1763783 , -0.19750185],
         [ 0.48794508,  0.06732991, -0.29097188],
         [ 0.4256317 ,  0.05175158, -0.27539355]],

        [[-1.630709  , -1.5683956 , -1.5683956 ],
         [-1.8799624 , -1.8799624 , -1.8799624 ],
         [-1.5995524 , -1.7553357 , -1.8799624 ],
         ...,
         [ 0.03617324, -0.5090686 , -1.0231538 ],
         [-0.02614012, -0.5869603 , -1.1010455 ],
         [ 0.0205949 , -0.52464694, -0.9919971 ]],

        [[-1.490504  , -1.5060823 , -1.5528173 ],
         [-1.630709  , -1.7709141 , -1.8799624 ],
         [-1.1166239 , -1.4593472 , -1.7553357 ],
         ...,
         [-0.04171846, -0.571382  , -1.1010455 ],
         [-0.01056178, -0.571382  , -1.1010455 ],
         [-0.18192351, -0.74274373, -1.2256722 ]],

        ...,

        [[ 1.360332  ,  0.7683552 , -0

In [10]:
def one_hot_encode_label(label_set):
    '''categorize the label set'''
    num_class = len(np.unique(label_set))
    return to_categorical(label_set, num_classes=num_class)
y_train = one_hot_encode_label(y_train)
y_test = one_hot_encode_label(y_test)
y_valid = one_hot_encode_label(y_valid)

In [12]:
# data augmentation

datagen = ImageDataGenerator(rotation_range=15, width_shift_range=0.1,
                            height_shift_range=0.1, horizontal_flip=True,
                             vertical_flip=True)
datagen.fit(X_train)

## Step 3 : Build a model

In [28]:
# base_hidden_units = 32 # number of filters
weight_decay = 1e-4 # l2 regularization 

model = Sequential() # initialize a sequential model

# layer 1 
model.add(Conv2D(base_hidden_units, kernel_size=3, padding='same', kernel_regularizer= regularizers.l2(weight_decay),
                input_shape=(X_train.shape[1:]), activation='relu'))
model.add(BatchNormalization())

# layer 2
model.add(Conv2D(base_hidden_units, kernel_size=3, padding='same', 
                kernel_regularizer=regularizers.l2(weight_decay), activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(rate=0.2))

# layer 3
model.add(Conv2D(base_hidden_units * 2, kernel_size= 3, padding='same',
kernel_regularizer=regularizers.l2(weight_decay)))
model.add(Activation('relu'))
model.add(BatchNormalization())

# layer 4
model.add(Conv2D(base_hidden_units * 2, kernel_size= 3, padding='same',
kernel_regularizer=regularizers.l2(weight_decay)))
model.add(Activation('relu'))
model.add(BatchNormalization())

# POOL + Dropout
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.3))

# layer 5
model.add(Conv2D(base_hidden_units * 4, kernel_size= 3, padding='same',
kernel_regularizer=regularizers.l2(weight_decay)))
model.add(Activation('relu'))
model.add(BatchNormalization())

# layer 6
model.add(Conv2D(base_hidden_units * 4, kernel_size= 3, padding='same',
kernel_regularizer=regularizers.l2(weight_decay)))
model.add(Activation('relu'))
model.add(BatchNormalization())

# POOL + Dropout
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))

model.add(Flatten())

model.add(Dense(10, activation='softmax'))

model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_14 (Conv2D)           (None, 32, 32, 32)        896       
_________________________________________________________________
batch_normalization_13 (Batc (None, 32, 32, 32)        128       
_________________________________________________________________
conv2d_15 (Conv2D)           (None, 32, 32, 32)        9248      
_________________________________________________________________
batch_normalization_14 (Batc (None, 32, 32, 32)        128       
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 16, 16, 32)        0         
_________________________________________________________________
dropout_6 (Dropout)          (None, 16, 16, 32)        0         
_________________________________________________________________
conv2d_16 (Conv2D)           (None, 16, 16, 64)       

## Step 4: Train the model

In [31]:
'''Note: this computer can't run the following, run on a GPU machine
'''

# batch_size = 128
# epochs = 125

# optimizer = optimizers.Adam(decay=1e-6, learning_rate=0.0001)
# modelchecker = ModelCheckpoint(filepath='model.weights_best.hdf5', verbose=1, save_weights_only=True)

# model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

# history = model.fit_generator(datagen.flow(X_train, y_train), epochs, batch_size,
#                              validation_data=(X_valid, y_valid), callbacks=[modelchecker],
#                              verbose=2)

Epoch 1/2
2/2 - 20s - loss: 4.6058 - accuracy: 0.0781 - val_loss: 11.8828 - val_accuracy: 0.1015

Epoch 00001: saving model to model.weights_best.hdf5
Epoch 2/2
2/2 - 19s - loss: 3.2092 - accuracy: 0.1719 - val_loss: 11.7302 - val_accuracy: 0.1014

Epoch 00002: saving model to model.weights_best.hdf5


## Step 5 : Evaluate model on test data

In [32]:
# Note : This won't produce the best result
score = model.evaluate(X_test, y_test, batch_size, verbose=2)
print(f'Accuracy :{score[1]}\n Loss : {score[0]}')

5000/5000 - 31s - loss: 11.8907 - accuracy: 0.1025
Accuracy :0.10249999910593033
 Loss : 11.890710830688477
